# Importar os dados


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import davies_bouldin_score
from adjustText import adjust_text
import plotly.io as pio
import plotly.graph_objects as go

file_path = "./Missing_Migrants_Global_Figures_allData.xlsx"
df = pd.read_excel(file_path)
display(df.head())

# Análise Descritiva dos Dados

In [ ]:
display(df.describe())

In [ ]:
# Selecionar apenas colunas numéricas para análise de correlação
numerical_cols = df.select_dtypes(include=np.number).columns

# Calcular a matriz de correlação
correlation_matrix = df[numerical_cols].corr()

# Gerar o heatmap
import seaborn as sns
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Heatmap de Correlação das Colunas Numéricas daora')
plt.show()

In [ ]:
cols_to_use = ['Migration Route', 'Total Number of Dead and Missing', 'Number of Survivors']
df_routes = df[cols_to_use].copy()


print("Rotas Nan: " + str(df_routes['Migration Route'].isnull().sum()))
print('% Rotas Nan: ' + str(df_routes['Migration Route'].isnull().sum() / len(df_routes) * 100))

print("\nNúmero de valores NaN na coluna 'Total Number of Dead and Missing':", end=' ')
print(str(df_routes['Total Number of Dead and Missing'].isnull().sum()))
print('% Rotas Nan: ' + str(df_routes['Total Number of Dead and Missing'].isnull().sum() / len(df_routes) * 100))

print("\nNúmero de valores NaN na coluna 'Number of Survivors':", end=' ')
print(str(df_routes['Number of Survivors'].isnull().sum()))
print('% Rotas Nan: ' + str(df_routes['Number of Survivors'].isnull().sum() / len(df_routes) * 100))

## Corrigindo valores

In [ ]:
# Apagar linhas com a rota de migração igual a NaN
df_routes.dropna(subset=['Migration Route'], inplace=True)

In [ ]:
# Converter colunas numéricas de "erro/NaN" com 0

df_routes['Number of Survivors'] = pd.to_numeric(df_routes['Number of Survivors'], errors='coerce').fillna(0)

In [ ]:
# Filtrar rotas por ocorrência 

# Contar ocorrências de cada rota de migração
route_counts = df_routes['Migration Route'].value_counts()

# Obter rotas que aparecem pelo menos 10 vezes
routes_to_keep = route_counts[route_counts >= 10].index
routes_to_exclude = route_counts[route_counts < 10].index

# Filtrar o banco de dados
df_filtered = df_routes[df_routes['Migration Route'].isin(routes_to_keep)].copy()

# Mostrar rotas mantidas com número de ocorrências
print("Rotas mantidas (ocorreram pelo menos 10 vezes):")
display(route_counts[route_counts >= 10].to_frame(name='Ocorrências'))

# Mostrar rotas excluídas com número de ocorrências
print("\nRotas excluídas (ocorreram menos de 10 vezes):")
display(route_counts[route_counts < 10].to_frame(name='Ocorrências'))


In [ ]:
# Analisando dados melhorados
display(df_filtered.describe())
# Fazendo boxplot para visualizar a distribuição dos dados
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.boxplot(x='Total Number of Dead and Missing', data=df_filtered)
plt.title('Distribuição de Mortes e Desaparecidos')
plt.subplot(1, 2, 2)
sns.boxplot(x='Number of Survivors', data=df_filtered)
plt.title('Distribuição de Sobreviventes')
plt.tight_layout()
plt.show()

# Fazendo histogramas para visualizar a distribuição dos dados
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(df_filtered['Total Number of Dead and Missing'], bins=30, kde=True
    , color='blue')
plt.title('Histograma de Mortes e Desaparecidos')
plt.subplot(1, 2, 2)
sns.histplot(df_filtered['Number of Survivors'], bins=30, kde=True, color='green')
plt.title('Histograma de Sobreviventes')
plt.tight_layout()
plt.show()


### Danger Ratio

In [ ]:
# Calcular o dominador para as taxas de letalidade
denominator = df_filtered['Total Number of Dead and Missing'] + df_filtered['Number of Survivors']

# Calcular a taxa de perigo para cada incidente
df_filtered['Danger Ratio'] = np.divide(
    df_filtered['Total Number of Dead and Missing'],
    denominator,
    out=np.zeros_like(df_filtered['Total Number of Dead and Missing'], dtype=float),
    where=(denominator != 0)
)

# Calcula o número total de pessoas envolvidas em cada rota
df_filtered['Total People Involved'] = denominator


In [ ]:
# Normalização dos dados

# Agrupamento dos dados
normalized_summary = df_filtered.groupby('Migration Route').agg(
    Mean_Danger_Ratio=('Danger Ratio', 'mean'),
    Mean_People_Involved=('Total People Involved', 'mean')
)
# Selecionando as características para escala
features_to_scale = normalized_summary[['Mean_Danger_Ratio', 'Mean_People_Involved']]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_to_scale)

# Modelos de IA

Foi escolhido dois modelos de agrupamento: K-means e Agrupamento Hierárquico

## Primeiro Modelo - K-means

In [ ]:
# Número de clusters esclhidos: 3
k_optimal = 3
kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
normalized_summary['Cluster'] = kmeans.fit_predict(scaled_features)

# Análise de resultados
cluster_profiles = normalized_summary.groupby('Cluster')[['Mean_Danger_Ratio', 'Mean_People_Involved']].mean().sort_values('Mean_Danger_Ratio')
cluster_profiles['Quantidade'] = normalized_summary.groupby('Cluster').size()

print('--- Análise de Clusters com Dados Normalizados (k=3) ---')
print("\nPerfil Médio de Cada Cluster:")
print(cluster_profiles)

# Criar os clusters
profile_map = {
    cluster_profiles.index[0]: "Rotas com alta taxa de perigo e poucas pessoas envolvidas",
    cluster_profiles.index[1]: "Rotas com taxa de perigo média e número médio de pessoas envolvidas",
    cluster_profiles.index[2]: "Rotas com baixa taxa de perigo e muitas pessoas envolvidas",
}
normalized_summary['Cluster_Profile'] = normalized_summary['Cluster'].map(profile_map)

print("\n--- Detalhes dos Clusters (k=3) ---")
for cluster_label, profile_name in profile_map.items():
    print(f"\n--- Cluster: {profile_name} ---")
    cluster_data = normalized_summary[normalized_summary['Cluster'] == cluster_label]
    print(f"Número de Rotas: {len(cluster_data)}")
    print("Perfil Médio:")
    display(cluster_profiles.loc[cluster_label].round(2))
    print("\nRotas neste Cluster:")
    display(cluster_data.sort_values('Mean_People_Involved', ascending=False))


#Tabela com cada uma das rotas, seus valores e seu cluster
print("--- Tabela Final de Clusterização (K-Means, k=3) ---")
final_table_kmeans = normalized_summary[['Mean_Danger_Ratio', 'Mean_People_Involved', 'Cluster_Profile']].copy()
final_table_kmeans.sort_values(by=['Cluster_Profile', 'Mean_Danger_Ratio'], ascending=[True, False], inplace=True)
display(final_table_kmeans)

Gráfico do Agrupamento não Hierárquico

In [ ]:

plt.figure(figsize=(10, 6))

# Obter os clusters únicos e criar um colormap manual
clusters = normalized_summary['Cluster'].unique()
colors = plt.cm.viridis(clusters / clusters.max())  # mapa de cores baseado nos clusters


# Plotar cada cluster separadamente para poder criar legenda
for cluster_id, color in zip(clusters, colors):
    cluster_data = normalized_summary[normalized_summary['Cluster'] == cluster_id]
    plt.scatter(
        cluster_data['Mean_Danger_Ratio'],
        cluster_data['Mean_People_Involved'],
        label=cluster_data['Cluster_Profile'].iloc[0],
        c=[color],
        s=50
    )


plt.xlabel('Mean Danger Ratio')
plt.ylabel('Mean People Involved')
plt.title('Clusterização de Rotas de Migração (K-Means, k=3 - Escala Original)')
plt.grid(True)


# Anotações com ajuste automático
texts = []
for i, route in enumerate(normalized_summary.index):
    x = normalized_summary['Mean_Danger_Ratio'].iloc[i]
    y = normalized_summary['Mean_People_Involved'].iloc[i]
    texts.append(plt.text(x, y, route, fontsize=8))

adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray', lw=0.5))

plt.show()


## Segundo Modelo - Hierárquico Bottom-Up (Aglomerativo)

In [ ]:
METHOD = 'ward'  # Método de ligação para o dendrograma
METRIC = 'euclidean'  # Métrica de distância para o dendrograma

# Gerar a matriz de ligação para o dendrograma
Z = linkage(scaled_features, method=METHOD, metric=METRIC)

# Definir summary_2d como uma cópia de normalized_summary
summary_2d = normalized_summary.copy()

# Criar e salvar o dendrograma
plt.figure(figsize=(15, 10))
plt.title('Dendrograma Hierárquico (Letalidade vs. Escala)')
plt.xlabel('Rotas Migratórias')
plt.ylabel('Distância')
dendrogram(
    Z,
    labels=summary_2d.index,
    leaf_rotation=90,
    leaf_font_size=10,
)
plt.tight_layout()


# Extrair 3 clusters para comparar com a análise de K-Means
n_clusters = 3
hc = AgglomerativeClustering(n_clusters=n_clusters, metric=METRIC, linkage=METHOD)
summary_2d['Cluster'] = hc.fit_predict(scaled_features)

# Análises de Clusters hierárquicos 
cluster_profiles = summary_2d.groupby('Cluster')[['Mean_Danger_Ratio', 'Mean_People_Involved']].mean().sort_values('Mean_Danger_Ratio')

print("\n--- Análise dos Clusters Hierárquicos (Letalidade vs. Escala) ---")
print("\nPerfil Médio de Cada Cluster:")
print(cluster_profiles)

# Criar nomes descritivos baseados nos perfis calculados
profile_map = {
    cluster_profiles.index[0]: "Rotas com taxa de perigo média e número médio de pessoas envolvidas",
    cluster_profiles.index[1]: "Rotas com alta taxa de perigo e poucas pessoas envolvidas",
    cluster_profiles.index[2]: "Rotas com baixa taxa de perigo e muitas pessoas envolvidas"
}
summary_2d['Cluster_Profile']= summary_2d['Cluster'].map(profile_map)


print("\n--- Detalhes dos Clusters Hierárquicos ---")
for cluster_label, profile_name in sorted(profile_map.items(), key=lambda item: cluster_profiles.loc[item[0]]['Mean_Danger_Ratio']):
    print(f"\n--- Cluster: {profile_name} ---")
    cluster_data = summary_2d[summary_2d['Cluster'] == cluster_label]
    print(f"Número de Rotas: {len(cluster_data)}")
    print("Perfil Médio:")
    display(cluster_profiles.loc[cluster_label].round(2))
    print("\nRotas neste Cluster:")
    display(cluster_data[['Mean_Danger_Ratio', 'Mean_People_Involved']].sort_values('Mean_People_Involved', ascending=False))


#Tabela com cada uma das rotas, seus valores e seu cluster
display("--- Tabela Final de Clusterização (Hierárquico, n=3) ---")
final_table_hc = summary_2d[['Mean_Danger_Ratio', 'Mean_People_Involved', 'Cluster_Profile']].copy()
final_table_hc.sort_values(by=['Cluster_Profile', 'Mean_Danger_Ratio'], ascending=[True, False], inplace=True)
display(final_table_hc)

# Resultados

In [ ]:
# Scores para modelo k-means
kmeans_labels = kmeans.labels_
silhouette_kmeans = metrics.silhouette_score(scaled_features, kmeans_labels, metric='euclidean')
db_kmeans = davies_bouldin_score(scaled_features, kmeans_labels)
print("Silhouette Score (K-means):", silhouette_kmeans)
print("Davies Bouldin Score (K-Means):", db_kmeans, "\n")

# Scores para modelo hierárquico
hc_labels = hc.labels_
silhouette_hc = metrics.silhouette_score(scaled_features, hc_labels, metric='euclidean')
db_hc = davies_bouldin_score(scaled_features, hc_labels)
print("Silhouette Score (Hierárquico):", silhouette_hc)
print("Davies Bouldin Score (Hieráquico):", db_hc)



In [ ]:
# Tabela cruzada entre os rótulos dos dois algoritmos
# Crie o DataFrame de comparação entre os clusters
comparison_df = pd.DataFrame({
    'KMeans_Cluster': normalized_summary['Cluster'],
    'Hierarchical_Cluster': summary_2d['Cluster']
}, index=normalized_summary.index)

confusion = pd.crosstab(comparison_df['KMeans_Cluster'], comparison_df['Hierarchical_Cluster'])

print("Matriz de comparação entre clusters KMeans e Hierárquico:")
display(confusion)

In [ ]:
grupo_diferente = (comparison_df['Hierarchical_Cluster'] == 0) & (comparison_df['KMeans_Cluster'] == 0)

print("Grupo diferente (KMeans 0, Hierárquico 0):")
display(comparison_df[grupo_diferente])

In [ ]:
pio.renderers.default = "browser"

# Definir cores para cada cluster
cluster_colors = {
    0: 'blue',
    1: 'orange',
    2: 'green'
}

# Exemplo: dicionário com coordenadas de origem/destino para cada rota
coords_arrumadas = {
    'Afghanistan to Iran': {'origem_lat': 34.5, 'origem_lon': 69.1, 'destino_lat': 32.0, 'destino_lon': 54.0},
    'Belarus-EU border': {'origem_lat': 53.9, 'origem_lon': 27.5, 'destino_lat': 53.1, 'destino_lon': 23.1},
    'Caribbean to US': {'origem_lat': 18.0, 'origem_lon': -66.0, 'destino_lat': 25.7, 'destino_lon': -80.2},
    'Central Mediterranean': {'origem_lat': 33.8, 'origem_lon': 13.0, 'destino_lat': 37.6, 'destino_lon': 12.3},
    'Darien': {'origem_lat': 7.8, 'origem_lon': -77.5, 'destino_lat': 9.0, 'destino_lon': -79.5},
    'Dominican Republic to Puerto Rico': {'origem_lat': 18.5, 'origem_lon': -69.9, 'destino_lat': 18.5, 'destino_lon': -66.1},
    'Eastern Mediterranean': {'origem_lat': 36.2, 'origem_lon': 36.1, 'destino_lat': 38.0, 'destino_lon': 23.7},
    'Eastern Route to/from EHOA': {'origem_lat': 3.5, 'origem_lon': 42.0, 'destino_lat': 15.6, 'destino_lon': 45.2},
    'English Channel to the UK': {'origem_lat': 50.9, 'origem_lon': 1.8, 'destino_lat': 51.1, 'destino_lon': 1.3},
    'Haiti to Dominican Republic': {'origem_lat': 18.5, 'origem_lon': -72.3, 'destino_lat': 18.5, 'destino_lon': -69.9},
    'Horn of Africa Route': {'origem_lat': 12.0, 'origem_lon': 44.0, 'destino_lat': 15.6, 'destino_lon': 32.5},
    'Iran to Türkiye': {'origem_lat': 37.0, 'origem_lon': 45.0, 'destino_lat': 39.0, 'destino_lon': 43.0},
    'Italy to France': {'origem_lat': 43.7, 'origem_lon': 7.3, 'destino_lat': 43.8, 'destino_lon': 7.1},
    'Northern Route from EHOA': {'origem_lat': 9.0, 'origem_lon': 38.7, 'destino_lat': 15.0, 'destino_lon': 30.0},
    'Route to Southern Africa': {'origem_lat': 0.5, 'origem_lon': 32.5, 'destino_lat': -25.9, 'destino_lon': 28.2},
    'Sahara Desert crossing': {'origem_lat': 15.0, 'origem_lon': 1.0, 'destino_lat': 27.0, 'destino_lon': 13.0},
    'Sea crossings to Mayotte': {'origem_lat': -12.8, 'origem_lon': 45.2, 'destino_lat': -12.7, 'destino_lon': 45.1},
    'Syria to Türkiye': {'origem_lat': 36.2, 'origem_lon': 37.1, 'destino_lat': 37.1, 'destino_lon': 36.8},
    'Türkiye-Europe land route': {'origem_lat': 41.0, 'origem_lon': 28.9, 'destino_lat': 42.5, 'destino_lon': 22.5},
    'US-Mexico border crossing': {'origem_lat': 32.5, 'origem_lon': -117.0, 'destino_lat': 25.7, 'destino_lon': -100.3},
    'Ukraine to Europe': {'origem_lat': 50.4, 'origem_lon': 30.5, 'destino_lat': 52.5, 'destino_lon': 13.4},
    'Venezuela to Caribbean': {'origem_lat': 10.5, 'origem_lon': -66.9, 'destino_lat': 12.0, 'destino_lon': -61.8},
    'Western Africa / Atlantic route to the Canary Islands': {'origem_lat': 21.0, 'origem_lon': -17.0, 'destino_lat': 28.1, 'destino_lon': -15.4},
    'Western Balkans': {'origem_lat': 42.0, 'origem_lon': 21.0, 'destino_lat': 45.0, 'destino_lon': 15.0},
    'Western Mediterranean': {'origem_lat': 35.0, 'origem_lon': -2.9, 'destino_lat': 37.0, 'destino_lon': -1.8}
}


# Montar DataFrame para plotar
plot_df = []
for route, row in normalized_summary.iterrows():
    if route in coords_arrumadas:
        data = coords_arrumadas[route]
        plot_df.append({
            'origem_lat': data['origem_lat'],
            'origem_lon': data['origem_lon'],
            'destino_lat': data['destino_lat'],
            'destino_lon': data['destino_lon'],
            'cluster': row['Cluster'],
            'Migration Route': route
        })
plot_df = pd.DataFrame(plot_df)

# Plotar no mapa
fig = go.Figure()
for _, row in plot_df.iterrows():
    fig.add_trace(go.Scattergeo(
        lon = [row['origem_lon'], row['destino_lon']],
        lat = [row['origem_lat'], row['destino_lat']],
        mode = 'lines',
        line = dict(width = 2, color = cluster_colors[row['cluster']]),
        name = row['Migration Route']
    ))

fig.update_layout(
    title = 'Rotas de Migração por Cluster',
    showlegend = False,
    geo = dict(
        scope = 'world',
        projection_type = 'natural earth',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)'
    )
)
fig.show()  
